##### SQL Agent - Talking to the DB

In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
llm=OpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [4]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=False # use verbose=True for detailed step-by-step output
)

In [5]:
agent_executor.run("What is the total RWA and EAD per SEGMENT in the RWA_122022 table")

'The total RWA and EAD per SEGMENT in the RWA_122022 table are: Corporate: 85075812.12498187 RWA, 156657332.34921697 EAD; Retail: 37806680.90001285 RWA, 165698955.81288758 EAD.'

In [6]:
agent_executor.run("What is the total EXPOSURE of all clients that changed their STATUS from Performing to Non Performing between RWA_122022 table and RWA_032023 table")

'The total EXPOSURE of all clients that changed their STATUS from Performing to Non Performing between RWA_122022 table and RWA_032023 table is 1,845,945.45.'

In [7]:
# agent_executor.run("What is the EXPOSURE of Client 'sfman997ux8036' in RWA_122022 table")

In [8]:
client = agent_executor.run("Read all the information for Client 'sfman997ux8036' in RWA_122022 table and output it in json format")

In [9]:
client

'[{"CLIENT_NUMBER": "sfman997ux8036", "ACCOUNT_NUMBER": "1324710203qiph20", "PRODUCT": "CREDIT_CARD", "STATUS": "Performing", "SEGMENT": "Retail", "EXPOSURE_DRAWN": 0.0, "EXPOSURE_UNDRAWN": 18728.13876, "EXPOSURE": 18728.13876, "EAD": 11004.65713, "EAD_COLL": 0.0, "PD": 0.0246518, "LGD": 0.3092, "SIZE": 42.58029755, "MATURITY": 1.15616438, "F_MORTGAGE": "N", "F_REVOLVING": "Y", "F_LARGE_FIN": "N", "RW": 0.245013691, "RWA": 2696.291657}]'

##### Function calling

##### Custom function calling

In [10]:
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import Tool

from rw_formula import RW_Calc

In [11]:
llm = OpenAI(temperature=0)

prompt = PromptTemplate(
    input_variables=['client'],
    template="what is the RW if the PD of {client} doubles"
)

llm_rw_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

RW = Tool(
    name="risk weight calculator",
    func=RW_Calc,
    description=
    """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
)

tools = [RW]

agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=False
)

In [12]:
agent.run(f"for the client information in {client} what is the RW if PD doubles")

'0.3950618648985617'